# AI Youtube Video Voice Over and script Generator

### Install Libraries

In [ ]:
!pip install --q duckduckgo_search google-generativeai soundfile gradio-Client sounddevice langchain-groq langchain-core elevenlabs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 0s (504 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123598 fil

### import necessary Libraries

In [ ]:
# import neccessary libraries
import os
from groq import Groq
from google.colab import userdata
from gradio_client import Client
from duckduckgo_search import DDGS
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import soundfile as sf
import sounddevice as sd
from gradio_client import Client
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from elevenlabs.client import ElevenLabs
from elevenlabs import play, save, stream, Voice, VoiceSettings

In [ ]:
def get_user_input():
  """Prompts the user for a video title."""
  title = input("Enter the title of your YouTube video: ")
  return title

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import os
from groq import Groq
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [ ]:
# Gemini Model
def search_and_summarize(search_query):
  results = DDGS().text(
      keywords=search_query,
      region='us-en',
      safesearch='off',
      timelimit='2m',
      max_results=5
  )
# Extract text content from search results
  text_content = [result['body'] for result in results]
  gemini =  genai.GenerativeModel('gemini-1.5-flash')
  summary = gemini.generate_content(
      "Summarize the following text: " + " ".join(text_content),
  )
  context=summary.text
  return context

In [ ]:
# Generating youtube script with Gemini
prompt='''

Generate a YouTube script voiceover based on the following title and context.  Make it sound like a natural conversation, with pauses and a conversational tone.

Title: {title}

Context: {context}

The voiceover should start with a strong introduction, followed by key points that highlight the main aspects of the topic, and conclude with a statement that leaves the viewer wanting to learn more.

Only provide the voiceover text, without any additional formatting or instructions.
'''
def generate_youtube_script(title, context):
  """Generates a YouTube script based on the title and summary."""
  gemini =  genai.GenerativeModel('gemini-1.5-flash')
  script = gemini.generate_content([prompt,title,context]
  )
  return script

In [ ]:
# Text to speech from third party host api available at Hugging face using MELLOTTS-ENGLISH Model
def text_to_speech(output):
    client = Client("mrfakename/MeloTTS")
    result = client.predict(
        text=output,
        speaker="EN-US",
        speed=1,
        language="EN",
        api_name="/synthesize"
    )
    data, samplerate = sf.read(result)
    output_filename = "output2.wav"
    sf.write(output_filename, data, samplerate)
    print(f"Audio saved to: {output_filename}")
    return output_filename

In [ ]:
# Main Function
def main():
  title = get_user_input()
  summary = search_and_summarize(title)
  script = generate_youtube_script(title, summary)
  output=script.text
  audio=text_to_speech(output)
  return audio
if __name__ == "__main__":
    main()

Enter the title of your YouTube video: why paris olympics costed so cheap to organise


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded as API: https://mrfakename-melotts.hf.space ✔
Audio saved to: output2.wav


In [ ]:
# search and summarize with Groq llm
def search_and_summarize(search_query):
    # Use DuckDuckGo search to get search results
    results = DDGS().text(
        keywords=search_query,
        region='us-en',
        safesearch='off',
        timelimit='2m',
        max_results=5
    )
    # Extract text content from search results
    text_content = [result['body'] for result in results]
    # Initialize Groq client and create a chat completion
    client = Groq()
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "user",
                "content": "Summarize the following text: " + " ".join(text_content)
            }
        ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    # Collect the summarized content
    summary = ""
    for chunk in completion:
        summary += chunk.choices[0].delta.content or ""
    return summary

In [1]:
# Generate youtube Script with Groq LLM
def generate_youtube_script(title, context):
    """Generates a YouTube script based on the title and summary."""
    llm = ChatGroq(temperature=0.1, model_name="llama-3.1-70b-versatile")
    system = "You are a skilled scriptwriter for YouTube videos."
    # Create the prompt using the custom template
    prompt_text = f"""
    Generate a YouTube script voiceover for one minute based on the following title and context. Make it sound like a natural conversation, with pauses and a conversational tone .

    Title: {title}

    Context: {context}

    The voiceover should start with a strong introduction, followed by key points that highlight the main aspects of the topic, and conclude with a statement that leaves the viewer wanting to learn more.

    Only provide the voiceover text, without any additional formatting or instructions.
    """
    # Create a chat prompt consisting of the system and human messages
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", prompt_text)])
    chain = prompt | llm
    response = chain.invoke({"text": prompt_text})
    script = response.content
    return script

In [ ]:
# Text to Speech Using the ELeven Labs API

ELEVEN_LABS_API_KEY = userdata.get('ELEVEN_LABS_API_KEY')
client = ElevenLabs(api_key=ELEVEN_LABS_API_KEY)
def text_to_speech(output):
  audio = client.generate(
   text= output,
   voice="Brian"
  )
  return save(audio, "outpu3.mp3")

Disclaimer : Audio File will be saved in your files you can download it from there